In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


# Locate and load the data file
df = pd.read_csv('titanic_preprocessed.csv')

In [2]:
dfX = df.loc[:, df.columns != 'Survived']
dfy = df.loc[:, df.columns == 'Survived'].values.ravel()

In [3]:
X = dfX.values
y = dfy

In [4]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

# 10-fold CV evaluation of a classifier
def eval_classifier(_clf, _X, _y):
    acc = []
    kf = StratifiedKFold(n_splits=10, shuffle=False, random_state=None)
    for train_index, test_index in kf.split(_X, _y):
        _clf.fit(_X[train_index], _y[train_index])
        y_pred = _clf.predict(_X[test_index])
        acc += [accuracy_score(_y[test_index], y_pred)]
    return np.array(acc)

In [5]:
counts = np.unique(y, return_counts=True)
NBpriors = [counts[1][0]/len(y), counts[1][1]/len(y)]

In [6]:
from sklearn.naive_bayes import GaussianNB

In [7]:
from sklearn.ensemble import RandomForestClassifier

In [8]:
acc = eval_classifier(GaussianNB(priors=NBpriors), X, y)
print(f'Naive Bayes CV accuracy={np.mean(acc):.2f} {chr(177)}{np.std(acc):.3f}')

# For reference
acc = eval_classifier(RandomForestClassifier(n_estimators=200, max_depth=5, random_state=None, n_jobs=4), X, y)
print(f'Random Forest CV accuracy={np.mean(acc):.2f} {chr(177)}{np.std(acc):.3f}')

Naive Bayes CV accuracy=0.45 ±0.018
Random Forest CV accuracy=0.83 ±0.032


In [9]:
from sklearn.tree import DecisionTreeClassifier

def weakDT_fit(_list_cols, _X, _y):
    Xs = _X[:,_list_cols]
    return DecisionTreeClassifier( max_depth = 2 ).fit(Xs, _y)

def weakDT_predict(_clf, _list_cols, _X):
    Xs = _X[:,_list_cols]
    return _clf.predict(Xs), _clf.predict_proba(Xs)

# Use _m features randomly selected, a total of M weak learners
def features_randomsubset(_M, _m, nensemble=1):
    from numpy.random import choice
    # returns a list of list of column choices - subset features
    return [choice(_M, _m, replace=False) for _ in range(nensemble)]

In [12]:
%%time

def eval_weak(_X, _y, _nensemble, _nfeatures):
    acc = []
    for j in range(_nensemble):
        # Keep subset features, columns same for a 10-fold
        cols = features_randomsubset(_X.shape[1], _nfeatures, nensemble=1)
        # 10-fold CV
        kf = StratifiedKFold(n_splits=10, shuffle=False, random_state=None)
        for train_index, test_index in kf.split(_X, _y):
            clf = weakDT_fit(cols[0], _X[train_index], _y[train_index])
            y_pred, y_prob = weakDT_predict(clf, cols[0], _X[test_index])
            acc += [accuracy_score(_y[test_index], y_pred)]
    #
    return np.array(acc)
    
# Measure individual weak learners performance
acc = eval_weak(X, y, 100, 5)

# Sanity
print(f'total #results={len(acc)}')

print(f'Weak learners average Acc= {np.mean(acc):.2f} {chr(177)}{np.std(acc):.3f}')

total #results=1000
Weak learners average Acc= 0.67 ±0.064
Wall time: 585 ms


In [16]:
# Generate numerous trained NB classifiers based on subset features
def ensembleDT_fit(_ensemble_cols, _X, _y):
    # the list of ensemble columns have a column list for every member of the ensemble
    nensemble = len(_ensemble_cols)
    # list of weak learners
    ensemble_clf = []
    for j in range(nensemble):
        ensemble_clf += [weakDT_fit(_ensemble_cols[j], _X, _y)]
    #
    return ensemble_clf

# Using learned ensemble predict the outcome with majority voting
def ensembleDT_predict(_ensemble_clf, _ensemble_cols, _Xtest):
    from collections import defaultdict
    nensemble = len(_ensemble_clf)
    assert nensemble==len(_ensemble_cols)  # Error check
    # weak learner predictions
    ypred_e, yprob_e = [], []
    for j in range(nensemble):
        res = weakDT_predict(_ensemble_clf[j], _ensemble_cols[j], _Xtest)
        ypred_e += [res[0]]
        yprob_e += [res[1]]  # score of the prediction
    # majority voting for each data point in _Xtest
    ypred = []
    for i in range(_Xtest.shape[0]):
        ypred_scores = defaultdict(float)
        for j in range(nensemble):
            for c, p in enumerate(yprob_e[j][i]):
                # a proper score is necessary
                ypred_scores[c] += p
        ix = max(ypred_scores.items(), key=lambda a: a[1])
        ypred += [ix[0]]
    #
    return np.array(ypred)

In [17]:
%%time

def eval_ensemble(_X, _y, _niter, _nensemble, _nfeatures):
    acc = []
    for i in range(_niter):
        # Keep subset features, columns same for a 10-fold
        cols = features_randomsubset(_X.shape[1], _nfeatures, nensemble=_nensemble)
        # 10-fold CV
        kf = StratifiedKFold(n_splits=10, shuffle=False, random_state=None)
        for train_index, test_index in kf.split(_X, _y):
            clf = ensembleDT_fit(cols, _X[train_index], _y[train_index])
            y_pred = ensembleDT_predict(clf, cols, _X[test_index])
            acc += [accuracy_score(_y[test_index], y_pred)]
    #
    return np.array(acc)

# Measure ensemble weak learners performance
acc = eval_ensemble(X, y, 10, 200, 7)

print(f'Ensemble learners average Acc= {np.mean(acc):.2f} {chr(177)}{np.std(acc):.3f}')

Ensemble learners average Acc= 0.77 ±0.033
Wall time: 9.02 s


In [18]:
corrs = np.array([np.correlate(X[:,j], y)[0] for j in range(X.shape[1])])

# Reverse sort, numpy array negation reverses the order
ranks = np.argsort((-corrs))

# Display top-9 and bot-5
rankings = [(f'{corrs[j]:.1f}', df.columns[j]) for j in ranks]

In [19]:
delcols = [(j, f'{corrs[j]:.1f}', df.columns[j]) for j in ranks if corrs[j]<=2]

In [20]:
# Column numbers to delete
dd = [d[0] for d in delcols]

# Drop those columns, axis=1
Xpp = np.delete(np.array(X, copy=True), dd, axis=1)

In [21]:
acc = eval_ensemble(Xpp, y, 10, 200, 11)
print(f'Ensemble learners average Acc= {np.mean(acc):.2f} {chr(177)}{np.std(acc):.3f}')

Ensemble learners average Acc= 0.82 ±0.027


In [22]:
# Number of features
M = Xpp.shape[1]

In [23]:
%%time

# Run an experiment for a full scale of number of features
valsF, accF, stdevF = np.arange(1,M+1), [], []
for nf in valsF:
    acc = eval_ensemble(Xpp, y, 10, 200, nf)
    accF += [np.mean(acc)]
    stdevF += [np.std(acc)]

Wall time: 7min 9s
